In [1]:
import lxw
import pandas as pd
import re

## 1. 代码编写

In [62]:
def Generate_Java_Code(task, model = ''):
    system_prompt = '''
# Profile
You are a Java Engineer responsible for writing Java code.

# Workflow
1. Users input the description of the task.
2. You generate the Java code.

# Constraint
Your output should be generated clean Java code, without any other content.

# Output
Your generated Java code.
'''

    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": task
        }
    ]

    if not model:
        model = lxw.gpt4_turbo
        
    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message.content

In [59]:
def Review_Java_Code(task, code, model = ''):
    system_prompt = '''
# Profile
You are a Java expert responsible for reviewing Java code to ensure its correctness, robustness, performance, security, and readability.

# Workflow
1. Users input coding tasks and the written code.
2. You carefully understand the the user's scenarios and task requirements and devise a plan to review the code, considering potential problems that may arise during the coding process
3. You carry out the plan and review the code step by step.
4. You summarize and organize the feedback from the code review.
5. If the code is good and has no issues, you reply '评审通过没有问题' at the [Summary] section.

# Constraint
Your output language should be Chinese.

# Output
Your process of understanding, thinking, and analysis.
[总结]
#1 <<Your first comment>>
#2 <<Your second comment>>
...
'''
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": '#Task\r\n' + task + '\r\nCode:\r\n' + code
        }
    ]
    if not model:
        model = lxw.gpt4_turbo
        
    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    content = response.choices[0].message.content
    index = content.find('[总结]')
    if index >= 0:
        return content[index + len('[总结]'):]
    return content
    

In [27]:
def Refine_Java_Code(task, code, comments, model = ''):
    system_prompt = '''
# Profile
You are a Java expert responsible refine Java code.

# Workflow
1. Users input the original task requirements, code written according to the requirements, and the code review comments from the code review board.
2. You refine and revise the code based on the review suggestions.

# Output
Your refined Java code and comments, without any other content.
'''
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": '#Task\r\n' + task + '\r\nCode:\r\n' + code + '\r\n Code Review Comments:\r\n' + comments
        }
    ]
    if not model:
        model = lxw.gpt4_turbo
        
    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message.content

#### 1.1 利率计算

你是一名Java开发人员。你负责编写代码完成每日根据利率结算利息任务。函数的声明如下：<br/>
public static long[] calculateInterest(long[] accountBalances, BigDecimal dailyInterestRate) <br/>
输入：<br/>
accountBalances：账户余额，精确到分。<br/>
dailyInterestRate：当日利率，精确到12位小数。<br/>
输出：计算好的利息。




In [63]:
task = '''
根据账户的余额与日利率结算利息。函数的声明如下：
public static long[] calculateInterest(long[] accountBalances, double dailyInterestRate)
输入：
accountBalances：账户余额，精确到分。
dailyInterestRate：当日利率，精确到12位小数。
输出：计算好的利息。
'''

In [64]:
code = Generate_Java_Code(task, model = lxw.gpt4_turbo_1106)
print(code)

```java
public static long[] calculateInterest(long[] accountBalances, double dailyInterestRate) {
    long[] interests = new long[accountBalances.length];
    for (int i = 0; i < accountBalances.length; i++) {
        double interest = accountBalances[i] * dailyInterestRate;
        interests[i] = Math.round(interest);
    }
    return interests;
}
```


In [65]:
feedback = Review_Java_Code(task, code, model = lxw.gpt4_turbo_0125)
print(feedback)


#1 代码逻辑正确，可以正确计算出每个账户的计息结果。
#2 考虑到金融类应用对精度的严苛要求，建议使用`BigDecimal`进行利息计算，以避免潜在的精度损失问题。
#3 增加边界条件及异常输入的检查，提高代码的健壮性和稳定性。


In [75]:
refined_code = Refine_Java_Code(task, code, feedback, model = lxw.gpt4_turbo_0125)
print(refined_code)

```java
import java.math.BigDecimal;
import java.math.RoundingMode;

public static long[] calculateInterest(long[] accountBalances, double dailyInterestRate) {
    long[] interests = new long[accountBalances.length];

    // 转换日利率为BigDecimal以确保精确计算
    BigDecimal rate = BigDecimal.valueOf(dailyInterestRate);

    for (int i = 0; i < accountBalances.length; i++) {
        // 只对正余额账户计算利息
        if (accountBalances[i] > 0) {
            // 将账户余额从“分”转换为“元”（即除以100）
            BigDecimal balanceInYuan = BigDecimal.valueOf(accountBalances[i]).divide(BigDecimal.valueOf(100));
            // 计算利息
            BigDecimal interest = balanceInYuan.multiply(rate).setScale(0, RoundingMode.HALF_UP);
            // 回转为“分”并存储结果
            interests[i] = interest.longValue() * 100;
        } else {
            // 负余额或零余额账户不计算利息
            interests[i] = 0;
        }
    }
    return interests;
}
```


In [76]:
feedback = Review_Java_Code(task, refined_code, model = lxw.gpt4_turbo_0125)
print(feedback)


#1 **精度处理注意事项**：代码使用了`BigDecimal`以确保金融计算的精度，这是非常适当的。然而，在将利息从元转回分时，通过`interest.longValue() * 100`的方式可能存在精度问题。正确的做法应该是先乘以100再使用`setScale`确保无小数部分，然后再转换成`long`类型，这样可以避免潜在的四舍五入错误。

#2 **利率转换逻辑的简化**：在计算利息时，建议将日利率直接应用于账户余额（已以分为单位），避免了将余额从分转换为元，计算后再从元转回分的过程。这样做可以简化逻辑，并可能提高性能。

#3 **异常处理**：方法的声明和当前实现中没有处理任何关于输入参数的异常情况，例如`null`值输入、`dailyInterestRate`不合理（负值或过高的值）等。建议添加对这些潜在问题的检测和处理逻辑，来提高代码的健壮性。

#4 **代码注释和可读性**：代码中的注释清晰地说明了各个步骤的目的，这对维护者理解代码逻辑很有帮助。不过，在一些关键变量转换处理上，考虑增加更详细的说明，以便让读者能够快速理解代码中的转换逻辑。

#5 **性能考量**：在处理大量账户余额数据时，应当注意循环内部的性能。目前的实现中，每次循环都会创建新的`BigDecimal`对象来计算利息，这对内存和处理速度都可能构成负担。若账户余额数量庞大，可以考虑对代码进行优化，以减少资源消耗。

总体而言，代码实现了基本功能，但在精度控制、异常处理和性能优化上还有改进空间。通过对上述建议进行调整，可以使代码更健壮、更高效，同时也提高了代码的安全性和可读性。


In [78]:
refined_code = Refine_Java_Code(task, code, feedback, model = lxw.gpt4_turbo_0125)
print(refined_code)

```java
import java.math.BigDecimal;

public static long[] calculateInterest(long[] accountBalances, double dailyInterestRate) {
    // Check for null input and invalid dailyInterestRate
    if (accountBalances == null || dailyInterestRate < 0) {
        throw new IllegalArgumentException("Invalid input");
    }

    long[] interests = new long[accountBalances.length];
    BigDecimal rate = BigDecimal.valueOf(dailyInterestRate);

    for (int i = 0; i < accountBalances.length; i++) {
        // Convert account balance to BigDecimal for precision
        BigDecimal balance = BigDecimal.valueOf(accountBalances[i]);
        // Calculate interest in BigDecimal then convert it to long (detailed precision handling)
        BigDecimal interest = balance.multiply(rate).setScale(0, BigDecimal.ROUND_HALF_UP);
        interests[i] = interest.longValue();
    }
    return interests;
}
```


In [84]:
feedback = Review_Java_Code(task, refined_code, model = lxw.gpt4_turbo_0125)
print(feedback)


#1 使用BigDecimal对货币计算进行精确处理，避免了精度丢失的问题。
#2 需要考虑对负利率的处理，这取决于具体业务场景。
#3 异常处理中，应包含更多的错误细节，以便调用者定位问题。
#4 在性能敏感的应用中，要注意使用BigDecimal可能带来的性能影响。


In [85]:
refined_code = Refine_Java_Code(task, refined_code, feedback, model = lxw.gpt4_turbo_0125)
print(refined_code)

```java
import java.math.BigDecimal;
import java.math.RoundingMode;

public static long[] calculateInterest(long[] accountBalances, double dailyInterestRate) {
    // Check for null input and verify dailyInterestRate is within expected range
    if (accountBalances == null) {
        throw new IllegalArgumentException("Account balances must not be null.");
    }
    if (dailyInterestRate < -1) { // Assuming -100% is the minimum possible daily interest rate
        throw new IllegalArgumentException("Daily interest rate cannot be less than -100%.");
    }

    long[] interests = new long[accountBalances.length];
    BigDecimal rate = BigDecimal.valueOf(dailyInterestRate);

    for (int i = 0; i < accountBalances.length; i++) {
        // Handling negative balance with precision
        BigDecimal balance = new BigDecimal(accountBalances[i]);

        // Calculate interest precisely. Use HALF_UP for rounding mode to mimic traditional rounding.
        BigDecimal interest = balance.multip

In [87]:
feedback = Review_Java_Code(task, refined_code, model = lxw.gpt4_turbo_0125)
print(feedback)


#1 此段代码在保证精度的同时，很好地处理了输入验证，使用 `BigDecimal` 来进行金额和利率的计算，这对于金融相关软件是非常重要的。
#2 代码考虑了负余额的情况，这是合理的，因为在某些金融系统中，账户可能会有透支。
#3 使用 `BigDecimal.setScale(0, RoundingMode.HALF_UP)` 来进行四舍五入至整数部分是适当的，它遵循了传统的四舍五入规则。
#4 检查日利率小于-1的逻辑是合理的，防止了异常数据对计算结果产生影响。
#5 注释中提到了关于性能的重要提示，这是必要的，因为 `BigDecimal` 的使用可能对性能有较大影响。在性能敏感的应用中，可能需要进一步的性能优化措施。
#6 代码风格清晰，有利于阅读理解，同时也包括了适当且有用的注释。

综上所述，这段代码是高质量的，根据当前提供的需求和代码样例，没有明显的缺陷或需要改进的地方。代码的设计和实现都考虑得非常周到。


### 1.2 中位数计算

选自LeetCode（困难）题目。<br>

给定两个大小分别为 m 和 n 的正序（从小到大）数组 nums1 和 nums2。请你找出并返回这两个正序数组的 中位数 。 \
算法的时间复杂度应该为 O(log (m+n)) 。

示例 1： \
输入：nums1 = \[1, 3\], nums2 = \[2\] \
输出：2

方法声明如下： \
class Solution { \
    public double findMedianSortedArrays(int[] nums1, int[] nums2) { \
    } \
}


In [88]:
task = '''
给定两个大小分别为 m 和 n 的正序（从小到大）数组 nums1 和 nums2。请你找出并返回这两个正序数组的 中位数 。
算法的时间复杂度应该为 O(log (m+n)) 。

示例 1：
输入：nums1 = [1, 3], nums2 = [2]
输出：2

方法声明如下：
class Solution {
    public double findMedianSortedArrays(int[] nums1, int[] nums2) {
    } 
}
'''

In [89]:
code = Generate_Java_Code(task, model = lxw.gpt4_turbo_0125)
feedback = Review_Java_Code(task, code, model = lxw.gpt4_turbo_0125)
N = 1
while N < 10 and feedback.find('评审通过') < 0:
    code = Refine_Java_Code(task, code, feedback, model = lxw.gpt4_turbo_0125)
    feedback = Review_Java_Code(task, code, model = lxw.gpt4_turbo_0125)
    N += 1

print(code)

```java
class Solution {
    public double findMedianSortedArrays(int[] nums1, int[] nums2) {
        if (nums1.length > nums2.length) {
            return findMedianSortedArrays(nums2, nums1);
        }
        
        int x = nums1.length;
        int y = nums2.length;
        int low = 0;
        int high = x;
        
        while (low <= high) {
            int partitionX = (low + high) / 2;
            int partitionY = (x + y + 1) / 2 - partitionX;

            int maxLeftX = (partitionX == 0) ? Integer.MIN_VALUE : nums1[partitionX - 1];
            int minRightX = (partitionX == x) ? Integer.MAX_VALUE : nums1[partitionX];

            int maxLeftY = (partitionY == 0) ? Integer.MIN_VALUE : nums2[partitionY - 1];
            int minRightY = (partitionY == y) ? Integer.MAX_VALUE : nums2[partitionY];

            if (maxLeftX <= minRightY && maxLeftY <= minRightX) {
                if ((x + y) % 2 == 0) {
                    return ((double)Math.max(maxLeftX, maxLeftY) + Math.min(

![](./images/self-refine-1.png)

## 2. 通用模板

In [5]:
def Generate_Generic(task, model = ''):
    system_prompt = '''
# Workflow
1. Users input the description of the task.
2. You generate the response.

# Constraint
Your output language should be Chinese.
'''

    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": task
        }
    ]

    if not model:
        model = lxw.gpt4_turbo
        
    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message.content

In [9]:
def Review_Generic(task, result, model = ''):
    system_prompt = '''
# Profile
You are a reviewer.

# Workflow
1. Users input task and result.
2. You carefully understand the the user's scenarios and task requirements and review the result step by step.
3. You summarize and organize the feedback.
4. If the result is good enough, you reply '评审通过没有问题'.

# Constraint
Your output language should be Chinese.
'''
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": '<Task>\r\n' + task + '</Task>\r\n' + '<Result>\r\n' + result + '</Result>'
        }
    ]
    if not model:
        model = lxw.gpt4_turbo
        
    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    content = response.choices[0].message.content
    return content

In [10]:
def Refine_Generic(task, result, feedback, model = ''):
    system_prompt = '''
# Profile
You are a refiner.

# Workflow
1. Users input the original task requirements, result to the requirements, and the review suggestions.
2. You refine and revise the result based on the review suggestions.
'''
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": 
                '<Task>\r\n' + task + '</Task>\r\n' + 
                '<Result>\r\n' + result + '</Result>\r\n' + 
                '<Review-Suggestions>\r\n' + feedback + '</Review-Suggestions>'
        }
    ]
    if not model:
        model = lxw.gpt4_turbo
        
    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message.content

### 2.1 内容总结（1）

In [96]:
task = '''在200字以内，总结一下水浒传的关键内容'''


In [97]:
result = Generate_Generic(task, model = lxw.gpt4_turbo_1106)
print(result)

《水浒传》是中国古代四大名著之一，作者是施耐庵，与罗贯中的《三国演义》、吴承恩的《西游记》、曹雪芹的《红楼梦》并称。这部小说通过对宋江、吴用等108位梁山好汉的叙述，展现了他们反抗封建官僚、贪官污吏的斗争。故事背景设定在北宋末年，以官逼民反、义士聚义等为线索，刻画了一个个栩栩如生、个性鲜明的人物形象，并且充分展现了当时社会的矛盾和冲突。全书中描写了很多激动人心的战斗场景，并通过梁山英雄们的故事，表达了人民对于正义、自由、平等的渴望和追求，以及对现实社会不公的批判。《水浒传》贯穿着深刻的社会意义，同时文笔细腻，情节丰富多变，是中国文学史上的一颗璀璨明珠。


In [99]:
feedback = Review_Generic(task, result, model = lxw.gpt4_turbo_1106)
print(feedback)

总体评价：
审查结果显示，上述总结有效地概述了《水浒传》的关键内容，涵盖了主要故事背景、主题和社会价值。文字流畅、信息准确，并妥当地将这部史诗般的作品浓缩成简短的概述。

具体反馈：
1. 作者提及：正确地指出《水浒传》的作者是施耐庵，并将其与其他三大名著进行了比较，体现出《水浒传》在中国文化中的重要地位。
2. 主要人物与主题：汇总中提到宋江、吴用等108位梁山好汉，以及他们对抗封建社会不公的斗争，体现了小说主要的内容要旨。
3. 故事背景与情节：正确指出故事发生在北宋末年，以反映民间矛盾和冲突，并点出了官逼民反和聚义抗争这些核心情节。
4. 社会意义：该总结强调了小说的社会意义，即反映民众对正义、自由、平等的渴求，以及对现存社会不公的批判。
5. 文学价值：提到了《水浒传》的文学特征，如细腻的文笔和丰富多变的情节，体现出对作品艺术成就的认可。

改进建议：
- 虽然该总结内容涵盖周全，但没有提及作品中著名的一些章节或者情节，比如“智取生辰纲”、“宋江投靠朝廷”等，这些对于体现小说特色同样重要。考虑到字数限制，可以在不影响其他要点的前提下适当提及。
- 尽管无需过多赘述，但若能简要提及作品中的文学手法，例如对比、象征等，也有助于增加评述的深度。

综上所述，该总结准确且条理清晰，对《水浒传》的理解适当。稍加润色和丰富，可使其更为完善。


In [102]:
refined_result = Refine_Generic(task, result, feedback, model = lxw.gpt4_turbo_1106)
print(refined_result)

《水浒传》是中国古代四大名著之一，由施耐庵所著。它通过宋江、吴用等108位反抗封建压迫的梁山好汉的故事，展示了他们对抗官僚、贪官的不屈斗争。设于北宋末年，这部小说以官逼民反和士人聚义作为主要线索，描绘了丰富的人物性格并反映了深刻的社会矛盾。它包括许多激动人心的情节如“智取生辰纲”，还有宋江投靠朝廷等转折点，展现了人民对公正、自由、平等的追求和对不正之风的强烈批判。《水浒传》不仅社会意义深远，其精妙的文笔和运用对比、象征等文学手法也使其成为中国文学的瑰宝。


### 2.2 内容总结（2）

In [14]:
task = '''在200字以内，总结一下《西游记》的关键内容。要富有文学性与艺术性，能够让人引发阅读《西游记》的愿望。'''

result = Generate_Generic(task, model = lxw.gpt4_turbo_0125)
feedback = Review_Generic(task, result, model = lxw.gpt4_turbo_0125)
N = 0
while N < 3 and feedback.find('评审通过') >= 0:
    result = Refine_Generic(task, result, feedback, model = lxw.gpt4_turbo_0125)
    N += 1
print(result)

<Result>
《西游记》是一部幻想世界的杰作，讲述了唐僧师徒四人西天取经的艰难历程。他们跨越重重难关，抗击各种妖魔鬼怪，展示了坚韧不拔的毅力和对佛法的虔诚追求。孙悟空的机智勇敢、猪八戒的憨厚可爱、沙僧的忠诚稳重，以及唐僧的慈悲为怀，共同绘制了一幅生动活泼、充满想象的壮丽画卷。这部作品不仅仅是一场奇幻的冒险，更是一次关于忠诚、勇气、智慧与修行的深刻探索。它引领我们穿越神话的迷雾，探求人性的光芒，体验精神的升华。每一个故事都是一场心灵的触动，每一个情节都饱含哲理。《西游记》是中华文化的瑰宝，它邀请每一个渴望冒险、寻求真知的读者，一起踏上这条通往西天的神奇旅程。</Result>


### 2.3 翻译任务

In [20]:
task = '''
将以下内容翻译并润色为中文。要求符合中文表达习惯，使用简单的词汇，优雅通顺。
Spending lots of time thinking about different possible future scenarios and their probabilities might be captivating, but I suggest an absolutely different approach — to try to build your prototype yourself. First, it’s rather challenging and fun. Second, it will help us to look at our work in a more structured way. Third, it will give us an opportunity to try in practice one of the most cutting-edge approaches — LLM agents.
'''

In [22]:
result = Generate_Generic(task, model = lxw.gpt4_turbo_0125)
feedback = Review_Generic(task, result, model = lxw.gpt4_turbo_0125)
N = 0
while N < 5 and feedback.find('评审通过') >= 0:
    result = Refine_Generic(task, result, feedback, model = lxw.gpt4_turbo_0125)
    N += 1
print(result)

花费大量时间沉思各种可能的未来情境和它们的可能性或许很有吸引力，但我建议一种完全不同的方法 —— 亲手试着构建你的原型。首先，这种做法不仅充满挑战，也极富趣味。其次，它能帮我们以更加有序的方式来审视我们的工作。第三，它为我们提供了一个机会，让我们能够实践一种最尖端的方法 —— LLM 代理。
